<a href="https://colab.research.google.com/github/jyanivaddi/ERA_V1/blob/master/session_20/S20_StableDiffusion_modular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git -C ERA_V1 pull

fatal: destination path 'ERA_V1' already exists and is not an empty directory.
Already up to date.


In [2]:
import os
import sys
sys.path.append('/content/ERA_V1/session_20')

In [3]:
%pip install -qq -U datasets transformers pyarrow==9.0.0
%pip install -qq --upgrade transformers==4.25.1 diffusers ftfy accelerate


In [4]:
%%capture
!sudo apt -qq install git-lfs
!git config --global credential.helper store

In [5]:
import sys
import torch
sys.path.append('/content/ERA_V1/session_20/')
from stable_diffusion import StableDiffusion
from utils import invert_loss, get_style_embeddings, show_images

In [6]:
style_files = [ None,
               '/content/ERA_V1/session_20/learned_embeds_style_0.bin',
               '/content/ERA_V1/session_20/learned_embeds_style_10.bin',
               '/content/ERA_V1/session_20/learned_embeds_11.bin',
               '/content/ERA_V1/session_20/learned_embeds_style_5.bin']
               #'/content/ERA_V1/session_20/learned_embeds_style_9.bin']

torch_device = "cuda" if torch.cuda.is_available() else "cpu"

In [7]:
stable_diffuser = StableDiffusion(torch_device)

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.19.layer_norm2.bias', 'text_projection.weight', 'vision_model.encoder.layers.8.self_attn.v_proj.bias', 'vision_model.embeddings.patch_embedding.weight', 'vision_model.encoder.layers.4.self_attn.out_proj.weight', 'vision_model.encoder.layers.17.self_attn.k_proj.bias', 'vision_model.encoder.layers.8.layer_norm1.bias', 'vision_model.encoder.layers.14.self_attn.v_proj.bias', 'vision_model.encoder.layers.9.self_attn.out_proj.weight', 'vision_model.encoder.layers.20.mlp.fc2.bias', 'vision_model.encoder.layers.10.mlp.fc2.weight', 'vision_model.encoder.layers.22.self_attn.k_proj.bias', 'vision_model.encoder.layers.10.self_attn.q_proj.bias', 'vision_model.encoder.layers.5.mlp.fc2.weight', 'vision_model.encoder.layers.16.mlp.fc2.weight', 'vision_model.encoder.layers.16.self_attn.out_proj.bias', 'vision_model.encoder.layers.0.layer_norm2.weight', 'visi

In [ ]:
outputs_1 = []
outputs_2 = []
seed_values = [1,2,3,4,5]
prompt = "An oil painting of a lion eating pizza"
#prompt = "An oil painting of a bear playing guitar"
#prompt = "A restaurant that looks like the TajMahal"
#custom_loss_fn = blue_loss
custom_loss_scale = 100.0
num_styles = len(style_files)
for cnt in range(num_styles):
    style_token_embedding = get_style_embeddings(style_files[cnt]) if style_files[cnt] is not None else None
    this_generated_img_1 = stable_diffuser.generate_image_with_custom_style(prompt,
                                                          style_token_embedding = style_token_embedding,
                                                          random_seed = seed_values[cnt],
                                                          custom_loss_fn = None)
    outputs_1.append(this_generated_img_1)
    this_generated_img_2 = stable_diffuser.generate_image_with_custom_style(prompt,
                                                          style_token_embedding = style_token_embedding,
                                                          random_seed = seed_values[cnt],
                                                          custom_loss_fn = invert_loss,
                                                          custom_loss_scale = custom_loss_scale)
    outputs_2.append(this_generated_img_2)


show_images(outputs_1)
show_images(outputs_2)




  0%|          | 0/30 [00:00<?, ?it/s]